In [15]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession

In [16]:
spark = (SparkSession.builder.appName("window_lead_lag_nulls")
         .master("yarn").getOrCreate())

print('app_id'.ljust(40), spark.sparkContext.applicationId)

25/07/15 18:55:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/07/15 18:55:16 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/07/15 18:55:16 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/07/15 18:55:16 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/07/15 18:55:16 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/07/15 18:55:16 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
25/07/15 18:55:16 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
25/07/15 18:55:16 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
25/07/15 18:55:16 WARN Utils: Service 'SparkUI' could not bind on port 4048. Attempting port 4049.
25/07/15 18:55:16 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploadi

app_id                                   application_1727681258360_0164


In [17]:
sql = """
   select * from values 
    (0, null), 
    (1, 'X'), 
    (2, null),
    (3, null),
    (4, 'Y'),
    (5, null) as (id, val)
"""
df = spark.sql(sql)
df.createOrReplaceTempView('tbl_with_nulls')
spark.table('tbl_with_nulls').show()

+---+----+
| id| val|
+---+----+
|  0|null|
|  1|   X|
|  2|null|
|  3|null|
|  4|   Y|
|  5|null|
+---+----+



In [35]:
sql = """
  select 
   id,
   val,
    LEAD(val, 1, 0) IGNORE NULLS over w lead_ignore_nulls,
    LEAD(val, 1, 0) over w lead,
    '' as ___,
   val,
    LAG(val, 1, 0)  IGNORE NULLS over w lag_ignore_nulls,
    LAG(val, 1, 0)  over w lag
  from tbl_with_nulls 
   window w as (partition by '1' order by id)
  order by id
"""
df = spark.sql(sql)
df.show()

+---+----+-----------------+----+---+----+----------------+----+
| id| val|lead_ignore_nulls|lead|___| val|lag_ignore_nulls| lag|
+---+----+-----------------+----+---+----+----------------+----+
|  0|null|                X|   X|   |null|               0|   0|
|  1|   X|                Y|null|   |   X|               0|null|
|  2|null|                Y|null|   |null|               X|   X|
|  3|null|                Y|   Y|   |null|               X|null|
|  4|   Y|                0|null|   |   Y|               X|null|
|  5|null|                0|   0|   |null|               Y|   Y|
+---+----+-----------------+----+---+----+----------------+----+



In [61]:
from pyspark.sql.functions import count, lit, col

df.groupBy('lead_ignore_nulls').agg(
     count(lit('')).alias('cnt_Y')
        )\
  .withColumn('cnt_nulls_before_Y', col('cnt_Y') - 1)\
  .filter('lead_ignore_nulls = \'Y\'')\
  .select('lead_ignore_nulls', 'cnt_nulls_before_Y')\
  .distinct()\
  .show()

+-----------------+------------------+
|lead_ignore_nulls|cnt_nulls_before_Y|
+-----------------+------------------+
|                Y|                 2|
+-----------------+------------------+



In [11]:
spark.stop()